## Guider mode focal plane plot
This is intended to plot the guider amps on the focal plane. \
It is mainly as a check of the star locations. \
Craig Lage - 21-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from matplotlib.backends.backend_pdf import PdfPages
from photutils.detection import DAOStarFinder
import pickle as pkl
from astropy.time import Time, TimeDelta

In [ ]:
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam

camera = LsstCam.getCamera()

detectors = [['R00', 'SG0'], ['R00', 'SG1'], 
          ['R04', 'SG0'], ['R04', 'SG1'], 
          ['R40', 'SG0'], ['R40', 'SG1'],
          ['R44', 'SG0'], ['R44', 'SG1']]
detName = 'R44_SG0'
detector = camera[detName]
bbox = detector.getBBox()
nx,ny = bbox.getDimensions()
lct = LsstCameraTransforms(camera,detName)
cfpX, cfpY = lct.ccdPixelToFocalMm(nx/2, ny/2, detName)
# Number of arcseconds off axis.
print(cfpX*100*0.2, cfpY*100*0.2)

# Plot the CCD locations

In [ ]:
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam

camera = LsstCam.getCamera()

detectors = [['R00', 'SG0'], ['R00', 'SG1'], 
          ['R04', 'SG0'], ['R04', 'SG1'], 
          ['R40', 'SG0'], ['R40', 'SG1'],
          ['R44', 'SG0'], ['R44', 'SG1']]
fig = plt.figure(figsize=(10,10))
ax = fig.subplots(1,1)
plt.suptitle("LSSTCam Guider Mode CCD check", fontsize=24)
for [raft, ccd] in detectors:
    detName = f"{raft}_{ccd}"
    for detector in camera:
        if detector.getName()== detName:
            break
    bbox = detector.getBBox()
    nx,ny = bbox.getDimensions()
    lct = LsstCameraTransforms(camera,detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, detName)
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, detName)
    width = urfpX - llfpX
    height = urfpY - llfpY
    rect = plt.Rectangle((llfpX, llfpY), width=width, height=height, \
                      color='red', fill=False)
    ax.add_artist(rect)
    ax.scatter(llfpX, llfpY, marker='o', s=50, color='blue')
    if raft in ['R04', 'R44']:
        ax.text(llfpX-70.0, llfpY+10.0, f"{raft}_{ccd}", color='red')
    else:
        ax.text(llfpX+10.0, llfpY-10.0, f"{raft}_{ccd}", color='red')
    
    for amp in detector.getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        nx,ny = bbox.getDimensions()
        llfpX,llfpY = lct.ampPixelToFocalMm(0, 0, ampName)
        urfpX, urfpY = lct.ampPixelToFocalMm(nx, ny, ampName)
        width = urfpX - llfpX
        height = urfpY - llfpY
        rect = plt.Rectangle((llfpX, llfpY), width=width, height=height, \
                          color='black', fill=False)
        ax.add_artist(rect)
        textX = (llfpX + urfpX) / 2.0
        textY = (llfpY + urfpY) / 2.0
        if detName in ['R00_SG1', 'R04_SG0', 'R40_SG0', 'R44_SG1']:
            ax.text(textX+1.5, textY, ampName, fontsize=5, ha='center', va='center', rotation=90)
        else:
            ax.text(textX, textY, ampName, fontsize=5, ha='center', va='center')

ax.set_xlim(-300, 300)
ax.set_ylim(-300,300)

plt.savefig("/home/c/cslage/u/LSSTCam/images/Guider_Mode_Position_Check_22Apr25.png")

In [ ]:
amp.

# Plot the CCD locations with detected stars

In [ ]:
camera = LsstCam.getCamera()

detectors = [['R00', 'SG0'], ['R00', 'SG1'], 
          ['R04', 'SG0'], ['R04', 'SG1'], 
          ['R40', 'SG0'], ['R40', 'SG1'],
          ['R44', 'SG0'], ['R44', 'SG1']]
fig = plt.figure(figsize=(10,10))
ax = fig.subplots(1,1)
for [raft, ccd] in detectors:
    detName = f"{raft}_{ccd}"
    for detector in camera:
        if detector.getName()== detName:
            break
    bbox = detector.getBBox()
    nx,ny = bbox.getDimensions()
    lct = LsstCameraTransforms(camera,detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, detName)
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, detName)
    width = urfpX - llfpX
    height = urfpY - llfpY
    #print(detName, llfpX, llfpY, urfpX, urfpY, width, height)
    rect = plt.Rectangle((llfpX, llfpY), width=width, height=height, \
                      color='red', fill=False)
    ax.add_artist(rect)
    
    for amp in detector.getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        nx,ny = bbox.getDimensions()
        llfpX,llfpY = lct.ampPixelToFocalMm(0, 0, ampName)
        urfpX, urfpY = lct.ampPixelToFocalMm(nx, ny, ampName)
        width = urfpX - llfpX
        height = urfpY - llfpY
        #print(ampName, llfpX, llfpY, urfpX, urfpY, width, height)
        rect = plt.Rectangle((llfpX, llfpY), width=width, height=height, \
                          color='black', fill=False)
        ax.add_artist(rect)
ax.set_xlim(-350, 350)
ax.set_ylim(-350,350)

infile = open('/home/c/cslage/u/Guider_Mode/MotionDict_19Apr25.pkl', 'rb')
motionDict = pkl.load(infile)
infile.close()
for key in motionDict.keys():
    data = motionDict[key]
    [X,Y] = data[0][1]
    ax.scatter(X, Y, marker='x', color='green')


In [ ]:
bbox.endX

# To read the data from the dictionary and plot

In [ ]:
infile = open('/home/c/cslage/u/Guider_Mode/MotionDict_19Apr25.pkl', 'rb')
motionDict = pkl.load(infile)
infile.close()

dayObs = 20250417
seqNum = 599
expId = int(f"{dayObs}{seqNum:05d}")

pdf = PdfPages(f"/home/c/cslage/u/Guider_Mode/Guider_Mode_Star_Motion_{expId}.pdf")
fig = plt.figure(figsize=(10,10))
for key in motionDict.keys():
    ax = fig.subplots(1,1)
    print(key)
    data = motionDict[key]
    ts = []
    xs = []
    ys = []
    for [t, (x, y)] in data:
        ts.append(Time(t, scale='utc').unix_tai)
        xs.append(x)
        ys.append(y)
    ts = np.array(ts)
    xs = np.array(xs)
    ys = np.array(ys)
    ts -= ts[0]
    xs = (xs - np.median(xs)) * 100.0 * 0.2
    ys = (ys - np.median(ys)) * 100.0 * 0.2
    ax.plot(ts, xs, color='blue', marker='x', label='X')
    ax.plot(ts, ys, color='red', marker='+', label='Y')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Median subtracted position (arcsec)')
    ax.set_title(f"{expId} {key} {data[0][0]}")
    ax.legend()
    pdf.savefig(fig)
    plt.clf()
pdf.close()



In [ ]:
jumps = [['R00_SG0', +1], 
 ['R00_SG1', -1],
 ['R04_SG0', 0],
 ['R04_SG1', -1],
 ['R40_SG0', 0],
 ['R40_SG1', 0],
 ['R44_SG0', +1],
 ['R44_SG1', -1]]